In [1]:
import pandas as pd
import re

In [2]:
def read_whatsapp_chat(file_path) -> pd.DataFrame:

    # Defining filtering patterns
    encryption_message = "Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more."
    media_pattern = "<Media omitted>"
    email_pattern = r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}'  # the 'r' tells python to read it as raw text so '\' will be treated as '\'
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    edited_message = "<This message was edited>"
    deleted_message = "You deleted this message"
    null_message = "null"
    created_group_message = "created group"
    added_you_to_group_message = "added you"
    tagging_pattern = r'@[\w]+'

    # Read the file
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Filter out the unwanted lines
    filtered_lines = []
    for line in lines:
        if (
            encryption_message not in line and 
            deleted_message not in line and
            null_message not in line and
            media_pattern not in line and
            created_group_message not in line and
            added_you_to_group_message not in line and
            not re.search(email_pattern, line) and 
            not re.search(url_pattern, line)
        ):
            # for the below, the core message is good but it has some noise associated which we are removing
            line = line.replace(edited_message, "").strip()
            line = re.sub(tagging_pattern, "", line).strip()
            filtered_lines.append(line)

    # normalize content
    content = '\n'.join(filtered_lines)

    # ios specific cleaning
    content = content.replace('\u202f', ' ')  # replacing narrow space char
    content = re.sub(r'\[(\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2}(?::\d{2})?\s?[APap][Mm])\]',r'\1', content)  # removing square brackets around timestamp
        
    # characters that tell how to read, left to right or right to left
    content = content.replace('\u200E', '').replace('\u200F', '')

    pattern = r'(\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2}(?::\d{2})?(?:\s?[APap][Mm])?)\s?(?:-|\~)?\s?(.*?): (.*?)(?=\n\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2}|$)'
    messages = re.findall(pattern, content, re.DOTALL)

    df = pd.DataFrame(messages, columns = ['timestamp', 'sender', 'message'])

    timestamps = []
    for timestamp in df['timestamp']:
        try:
            timestamp = pd.to_datetime(timestamp, format='mixed', errors='coerce')
            
        except Exception as e:
            print(f'Error prsing timestamp \'{timestamp}\': {e}')
            timestamp = pd.NaT # converting time format errors that we could not prase into a "Not a Time" value

        timestamps.append(timestamp)

    df['timestamp'] = timestamps
    return df

In [3]:
from pathlib import Path

all_chats = {}
data_directory = Path('../data/private')

for file in data_directory.glob('*.txt'):
    file_name = file.stem
    all_chats[file_name] = read_whatsapp_chat(file)

Now `all_chats` is dictionary containing the cleaned texts of each file in a DataFrame format.

In [14]:
text_sequence = ""
for file_name in all_chats.keys():
    text_sequence += " ".join(all_chats[file_name]['message'].values)

len(text_sequence)

1185064

Now `text_sequence` is a long sequence of all texts

In [15]:

with open("../data/private/output/combined_text.txt", "w", encoding="utf-8") as f:
    f.write(text_sequence)